In [1]:
import scipy.io as sio
from scipy.misc import imread
from preprocess.normalize import preprocess_signature
import tf_signet
from tf_cnn_model import TF_CNNModel
import tensorflow as tf
import numpy as np
import pandas as pd
import sys
import os
import scipy.io
from find_largest_image import find_largest
import tqdm 
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from xgboost import XGBClassifier
import random
from numpy.random import choice

c:\users\mert\anaconda3\envs\ml_env\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [2]:
data_folder = 'C:\\Users\\Mert\\Documents\\GitHub\\sigver_bmg\\data\\downloaded_pp_features\\gpds_signet\\exploitation'

In [3]:
data_f = pd.DataFrame(columns=['user_id','sig_id','fakeness'])
visual_f = pd.DataFrame()

In [4]:
datalist = os.listdir(data_folder)

In [5]:
for data in tqdm.tqdm(datalist):
    if not(data.endswith('.mat')): continue
    if (data.split('.')[0].split('_')[0] == 'forg'): fakeness = True
    else: fakeness = False
    user_id = int(data.split('.')[0].split('_')[-1])
    matfile = sio.loadmat(os.path.join(data_folder, data))
    for row in range(matfile['features'].shape[0]):
        data_f = data_f.append(pd.DataFrame([[user_id, int(row+1), fakeness]], columns=['user_id', 'sig_id', 'fakeness']))
        visual_f = visual_f.append(pd.DataFrame(np.expand_dims(matfile['features'][row], 1).T))

100%|████████████████████████████████████████████████████████████████████████████████| 602/602 [09:04<00:00,  1.11it/s]


In [6]:
data_f = data_f.reset_index().drop(columns=['index'])
visual_f = visual_f.reset_index().drop(columns=['index'])

In [7]:
visual_f.shape

(16105, 2048)

In [8]:
data_f.shape

(16105, 3)

In [9]:
data_f.to_csv(os.path.join(data_folder,'data_features.csv'), index=False)
visual_f.to_csv(os.path.join(data_folder,'visual_features.csv'), index=False)

In [10]:
# data_f.loc[data_f['fakeness']==False]

In [11]:
data_f_gen = data_f.loc[data_f['fakeness']==False]
data_f_fake = data_f.loc[data_f['fakeness']==True]
data_f_gen_14 = data_f_gen.loc[data_f_gen['sig_id'].isin(np.arange(1,15))]
data_f_gen_valid_10 = data_f_gen.loc[data_f_gen['sig_id'].isin(np.arange(15,25))]

In [13]:
scores_sum = 0
run_count = 0
for user_id in tqdm.tqdm(data_f['user_id'].unique(), ascii=True):
    y_train = data_f_gen['user_id']==user_id
    X_train = visual_f.loc[y_train.index]
    # clf = XGBClassifier(tree_method='gpu_hist',verbosity=2,predictor='gpu_predictor', scale_pos_weight=8946/14)
    clf = SVC(class_weight='balanced', gamma='scale')
    clf.fit(X_train, y_train)
    y_valid_fakes = data_f_fake.loc[data_f_fake['user_id']==user_id]
    y_valid = len(y_valid_fakes.index)*[False]
    X_valid = visual_f.loc[y_valid_fakes.index]
    score = clf.score(X_valid,y_valid)
    # print(score)
    scores_sum += score
    run_count += 1
    y_valid_gens = data_f_gen_valid_10.loc[data_f_gen_valid_10['user_id']==user_id]
    y_valid = len(y_valid_gens.index)*[True]
    X_valid = visual_f.loc[y_valid_gens.index]
    score = clf.score(X_valid,y_valid)
    # print(score)
    scores_sum += score
    run_count += 1
    y_valid_rand_fakes = data_f_gen_14.loc[choice(data_f_gen_14.loc[data_f_gen_14['user_id']!=3].index,10)]
    y_valid = len(y_valid_rand_fakes.index)*[False]
    X_valid = visual_f.loc[y_valid_rand_fakes.index]
    score = clf.score(X_valid,y_valid)
    # print(score)
    scores_sum += score
    run_count += 1
    # print(scores_sum/run_count)
print(scores_sum/run_count)

100%|################################################################################| 300/300 [09:09<00:00,  1.59s/it]


0.9647001235035702


In [14]:
y_valid_fakes = data_f_fake.loc[data_f_fake['user_id']==user_id]
y_valid = len(y_valid_fakes.index)*[False]
X_valid = visual_f.loc[y_valid_fakes.index]

In [64]:
y_valid

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False]